# Version 11
**2021/08/31** *Metadata*
- deleted all usage of metadata and set metadata atribute (used for categories) for _Column and _DataFrame
- updated code with the usage of `self._col.df.some_function()` instead of metadata
- needet to correct how categorials and Arrow Dictionaries are handled in `dtype` and `dtype_from_vaexdytpe()` method

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%run vaex_implementation_v11.py

### Test data

In [3]:
x1 = np.array([True, True, False])
y1 = np.array([1, 2, 0])
z1 = np.array([9.2, 10.5, 11.8])
df1 = vaex.from_arrays(x=x1, y=y1, z=z1)
df1

#,x,y,z
0,True,1,9.2
1,True,2,10.5
2,False,0,11.8


In [4]:
df1.__dataframe__().metadata
df1.__dataframe__().get_column_by_name('x')._col.dtype
df1.__dataframe__().get_column_by_name('x').dtype
df1.__dataframe__().get_column_by_name('x').get_data_buffer()

{}

bool

(<_DtypeKind.BOOL: 20>, 8, '|b1', '|')

(VaexBuffer({'bufsize': 3, 'ptr': 2354358347488, 'device': 'CPU'}),
 (<_DtypeKind.BOOL: 20>, 8, '|b1', '|'))

In [5]:
_from_dataframe_to_vaex(df1.__dataframe__())

#,x,y,z
0,True,1,9.2
1,True,2,10.5
2,False,0,11.8


### Test data 2

In [6]:
indices = pa.array([0, 1, 2, 1, 2])
dictionary = pa.array(['foo', 'bar', 'baz'])
dict_array = pa.DictionaryArray.from_arrays(indices, dictionary)

the_df = vaex.from_arrays(
    numpy_int=np.array([1, 2, 3, 4, 0]), # Numpy int
    numpy_float=np.array([1.5, 2.5, 3.5, 4.5, 0]), # Numpy float
    numpy_bool=np.array([True, False, True, True, True]), # Numpy bool
    
    numpy_int_m=np.ma.array([1, 2, 3, 4, 0], mask=[0, 0, 0, 1, 1], dtype=int), # Numpy masked int
    numpy_float_m=np.ma.array([1.5, 2.5, 3.5, 4.5, 0], mask=[False, True, True, True, False], dtype=float), # Numpy masked float
    numpy_bool_m=np.ma.array([True, False, True, True, True], mask=[1, 0, 0, 1, 0], dtype=bool), # Numpy masked bool
    
    arrow_int = pa.array([0, 1, 2, 3, 0]), # Arrow integer
    arrow_float = pa.array([0.5, 1.5, 2.5, 3.5, 0.5]), # Arrow integer
    arrow_bool = pa.array([True, False, True, False, True]), # Arrow integer
    
    arrow_int_m = pa.array([0, 1, 2, None, 0], mask=np.array([0, 0, 0, 1, 1], dtype=bool)), # Arrow masked integer
    arrow_float_m = pa.array([0.5, 1.5, 2.5, None, 0.5], mask=np.array([0, 0, 0, 1, 0], dtype=bool)), # Arrow masked integer
    arrow_bool_m = pa.array([True, False, True, None, True], mask=np.array([0, 0, 1, 1, 0], dtype=bool)), # Arrow masked integer
    
    arrow_dict = pa.DictionaryArray.from_arrays(pa.array([0, 1, 2, 0, 1]), pa.array(['aap', 'noot', 'mies'])), # arrow dictionary
    arrow_dict_m = pa.DictionaryArray.from_arrays(pa.array([0, 1, 2, 0, 1], mask=np.array([0, 1, 1, 0, 0], dtype=bool)), pa.array(['aap', 'noot', 'mies'])) # arrow dictionary masked
)

In [7]:
the_df

#,numpy_int,numpy_float,numpy_bool,numpy_int_m,numpy_float_m,numpy_bool_m,arrow_int,arrow_float,arrow_bool,arrow_int_m,arrow_float_m,arrow_bool_m,arrow_dict,arrow_dict_m
0,1,1.5,True,1,1.5,--,0,0.5,True,0,0.5,True,aap,aap
1,2,2.5,False,2,--,False,1,1.5,False,1,1.5,False,noot,--
2,3,3.5,True,3,--,True,2,2.5,True,2,2.5,--,mies,--
3,4,4.5,True,--,--,--,3,3.5,False,--,--,--,aap,aap
4,0,0,True,--,0.0,True,0,0.5,True,--,0.5,True,noot,noot


In [8]:
_from_dataframe_to_vaex(the_df.__dataframe__())

#,numpy_int,numpy_float,numpy_bool,numpy_int_m,numpy_float_m,numpy_bool_m,arrow_int,arrow_float,arrow_bool,arrow_int_m,arrow_float_m,arrow_bool_m,arrow_dict,arrow_dict_m
0,1,1.5,True,1,1.5,--,0,0.5,True,0,0.5,True,aap,aap
1,2,2.5,False,2,--,False,1,1.5,False,1,1.5,False,noot,--
2,3,3.5,True,3,--,True,2,2.5,True,2,2.5,--,mies,--
3,4,4.5,True,--,--,--,3,3.5,False,--,--,--,aap,aap
4,0,0,True,--,0.0,True,0,0.5,True,--,0.5,True,noot,noot


In [9]:
the_df.__dataframe__().metadata
the_df.__dataframe__().get_column_by_name('arrow_dict').dtype
the_df.__dataframe__().get_column_by_name('arrow_dict')._col.dtype
the_df.__dataframe__().get_column_by_name('arrow_dict_m').dtype
the_df.__dataframe__().get_column_by_name('arrow_dict_m')._col.dtype
the_df.__dataframe__().get_column_by_name('numpy_int').dtype
the_df.__dataframe__().get_column_by_name('numpy_int')._col.dtype

{}

(<_DtypeKind.CATEGORICAL: 23>, 64, 'u', '=')

dictionary<values=string, indices=int64, ordered=0>

(<_DtypeKind.CATEGORICAL: 23>, 64, 'u', '=')

dictionary<values=string, indices=int64, ordered=0>

(<_DtypeKind.INT: 0>, 32, '<i4', '=')

int32

In [10]:
col = the_df.__dataframe__().get_column_by_name('arrow_dict')
codes_buffer, codes_dtype = col.get_data_buffer()
codes_buffer, codes_dtype

(VaexBuffer({'bufsize': 40, 'ptr': 7847446320576, 'device': 'CPU'}),
 (<_DtypeKind.INT: 0>, 64, '<i8', '='))

### Test data 3

In [11]:
# Numeric data with ordinal_encode and categorize test
colors = ['red', 'blue', 'green', 'blue']
ds = vaex.from_arrays(
    colors=colors, 
    year=[2012, 2013, 2015, 2019], 
    weekday=[0, 1, 4, 6], 
    x = [1, 2, 3, 4], 
    y = [1.5, 2.5, 3.5, 4.5])
df = ds.ordinal_encode('colors', ['red', 'green', 'blue'])
df = df.categorize('year', min_value=2012, max_value=2019)
df = df.categorize('weekday', labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
df

#,year,weekday,x,y,colors
0,2012,0,1,1.5,0
1,2013,1,2,2.5,2
2,2015,4,3,3.5,1
3,2019,6,4,4.5,2


In [12]:
df.__dataframe__().get_column_by_name('colors').dtype
df.__dataframe__().get_column_by_name('colors')._col.dtype
df.__dataframe__().get_column_by_name('year').dtype
df.__dataframe__().get_column_by_name('year')._col.dtype
df.__dataframe__().get_column_by_name('weekday').dtype
df.__dataframe__().get_column_by_name('weekday')._col.dtype

(<_DtypeKind.CATEGORICAL: 23>, 64, 'u', '=')

uint64

(<_DtypeKind.CATEGORICAL: 23>, 64, 'u', '=')

int32

(<_DtypeKind.CATEGORICAL: 23>, 64, 'u', '=')

int32

In [13]:
_from_dataframe_to_vaex(df.__dataframe__())

#,year,weekday,x,y,colors
0,2012,Mon,1,1.5,red
1,2013,Tue,2,2.5,blue
2,2015,Fri,3,3.5,green
3,2019,Sun,4,4.5,blue


In [14]:
col1 = df.__dataframe__().get_column_by_name('colors')
codes_buffer1, codes_dtype1 = col1.get_data_buffer()
codes_buffer1, codes_dtype1

(VaexBuffer({'bufsize': 32, 'ptr': 2354358341648, 'device': 'CPU'}),
 (<_DtypeKind.UINT: 1>, 64, '<u8', '='))

# Unit tests    

In [15]:
the_df2 = _from_dataframe_to_vaex(the_df.__dataframe__())

assert the_df['numpy_float'].tolist() == the_df2['numpy_float'].tolist()
assert the_df['arrow_int'].tolist() == the_df2['arrow_int'].tolist()
assert the_df['arrow_dict_m'].tolist() == the_df2['arrow_dict_m'].tolist()
assert the_df2.__dataframe__().get_column_by_name('numpy_float').null_count == 0
assert the_df2.__dataframe__().get_column_by_name('arrow_int').null_count == 0
assert the_df2.__dataframe__().get_column_by_name('arrow_dict_m').null_count == 2
assert the_df['numpy_float'].dtype == the_df2['numpy_float'].dtype
assert the_df['arrow_int'].dtype == the_df2['arrow_int'].dtype
assert isinstance(the_df2.__dataframe__().get_column_by_name('arrow_dict_m')._col.values, (pa.DictionaryArray))
assert the_df['arrow_dict_m'].values.type == the_df2['arrow_dict_m'].values.type

assert the_df2.__dataframe__().num_chunks() == 1
assert the_df2.__dataframe__().get_chunks()[0]._df.numpy_float.tolist() == the_df2.numpy_float.tolist()
assert the_df2.__dataframe__().get_column_by_name('numpy_float').num_chunks() == 1
assert the_df2.__dataframe__().get_column_by_name('numpy_float')._col.tolist() == the_df2.numpy_float.tolist()

col2 = the_df2.__dataframe__().get_column_by_name('arrow_dict')
assert col2.dtype[0] == 23



df2 = _from_dataframe_to_vaex(df.__dataframe__())

assert df2['year'].tolist() == [2012, 2013, 2015, 2019]
assert df2['weekday'].tolist() == ['Mon', 'Tue', 'Fri', 'Sun']
assert df2['colors'].tolist() == ['red', 'blue', 'green', 'blue']
assert df2.__dataframe__().get_column_by_name('year').null_count == 0
assert df2.__dataframe__().get_column_by_name('weekday').null_count == 0
assert df2.__dataframe__().get_column_by_name('colors').null_count == 0
assert df['year'].dtype == df2['year'].values.indices.type
assert df['weekday'].dtype == df2['weekday'].values.indices.type
assert df['colors'].dtype == df2['colors'].values.indices.type

assert df2.__dataframe__().num_chunks() == 1
assert df2.__dataframe__().get_column_by_name('year').num_chunks() == 1

In [16]:
df2.__dataframe__().get_column_by_name('year').dtype
df2.__dataframe__().get_column_by_name('weekday').dtype
df2.__dataframe__().get_column_by_name('colors').dtype

(23, 64, '|O', '=')

(23, 64, '|O', '=')

(23, 64, '|O', '=')